In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

MODEL_NUMBER = 5

    
# seed so random stuff isnt random
SEED = 100

# transforms
transform = transforms.Compose([

    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

dataset = torchvision.datasets.EuroSAT(root='./data', download=True, transform=transform)

# split into train, test
# 80% train, 20% test 
train_ratio = 0.8
test_ratio = 0.2


total_size = len(dataset)
train_size = int(total_size * train_ratio)
test_size = total_size - train_size


train_dataset, test_dataset = random_split(
    dataset, [train_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"train: {len(train_dataset)}")
print(f"test size: {len(test_dataset)}")

train: 21600
test size: 5400


In [13]:
import torchvision.models as models

# resnet 18 model
model = models.resnet18(weights=None)


# classifier head with 10 outputs
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 10)

print(f"Model: ResNet-18 (no pretrained weights)")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")


Model: ResNet-18 (no pretrained weights)
Total parameters: 11181642


In [9]:

# dataloaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=128, 
    shuffle=True,
    pin_memory=True, # for use on gpu
    num_workers=4 # arbitrarily chosen. some sources recommend 4 per gpu
)


test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=128, 
    shuffle=False,
    pin_memory=True,
    num_workers=4
)

In [10]:

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    model.to(device, non_blocking=True) # according to pytorch docs we can do this if memory is pinned
else:  
    model.to(device)
    


In [11]:
def train_one_epoch():
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

def test_one_epoch():
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [12]:
# tensorboard setup
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/train_{}_{}'.format(MODEL_NUMBER, timestamp))

checkpoint_dir = f'checkpoints/train_{MODEL_NUMBER}_{timestamp}'
os.makedirs(checkpoint_dir, exist_ok=True)

EPOCHS = 200

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    


    train_loss, train_acc = train_one_epoch()
    
    test_loss, test_acc = test_one_epoch()
    
    print(f'train loss: {train_loss:.4f} | train acc: {train_acc:.2f}%')
    print(f'test loss:  {test_loss:.4f} | test acc:  {test_acc:.2f}%')
    
    # log data
    writer.add_scalar('loss/1_train', train_loss, epoch + 1)
    writer.add_scalar('loss/2_test', test_loss, epoch + 1)
    writer.add_scalar('accuracy/1_train', train_acc, epoch + 1)
    writer.add_scalar('accuracy/2_test', test_acc, epoch + 1)
    writer.flush()
    
    # checkpoint every 20 epochs
    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        print(f'checkpoint saved: {checkpoint_path}')
    
    print()

# save final model
final_checkpoint_path = os.path.join(checkpoint_dir, f'model_final.pt')
torch.save(model.state_dict(), final_checkpoint_path)
print(f'final checkpoint saved: {final_checkpoint_path}')

writer.close()
print('Complete')

Epoch 1/200
train loss: 1.1723 | train acc: 56.91%
test loss:  2.1007 | test acc:  37.00%

Epoch 2/200
train loss: 0.9142 | train acc: 66.61%
test loss:  4.0525 | test acc:  29.70%

Epoch 3/200
train loss: 0.7958 | train acc: 71.39%
test loss:  1.9774 | test acc:  40.98%

Epoch 4/200
train loss: 0.7113 | train acc: 74.82%
test loss:  3.1684 | test acc:  33.50%

Epoch 5/200
train loss: 0.6283 | train acc: 77.60%
test loss:  1.0770 | test acc:  62.06%

Epoch 6/200
train loss: 0.5594 | train acc: 80.14%
test loss:  0.8743 | test acc:  68.11%

Epoch 7/200
train loss: 0.5066 | train acc: 82.13%
test loss:  1.4217 | test acc:  52.80%

Epoch 8/200
train loss: 0.4901 | train acc: 82.48%
test loss:  0.8937 | test acc:  70.28%

Epoch 9/200
train loss: 0.4267 | train acc: 85.28%
test loss:  1.8681 | test acc:  58.37%

Epoch 10/200
train loss: 0.4156 | train acc: 85.34%
test loss:  0.7403 | test acc:  74.19%

Epoch 11/200
train loss: 0.3901 | train acc: 86.19%
test loss:  1.1543 | test acc:  63.54